In [1]:
%load_ext autoreload
%autoreload 2

from dash import Dash, html, dcc, callback_context
import plotly.express as px
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import callbacks
import process_data


import warnings
warnings.filterwarnings("ignore", category=UserWarning)


engagements_raw, demographics_raw, followers_raw, top_posts_raw = process_data.read_data()
engagements, engagements_by_day = process_data.preprocess_engagements(engagements_raw)
demographics = process_data.preprocess_demographics(demographics_raw)
followers, total_followers, monthly_followers = process_data.preprocess_followers(followers_raw)
top_posts_preview_with_topics = process_data.preprocess_topposts(top_posts_raw)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'LinkedIn Analytics'


app.layout = html.Div([
        
        html.Img(
            src='/assets/img/Logo1.png',
            className='app-logo'
        ),
        
        html.H1(
            "LinkedIn Analytics Dashboard",
            className='title'
        ),
        dcc.Markdown(
            '**Choose a date range:** ',
            className='date-cta',
            ),
        dcc.DatePickerRange(
            id='date-picker-range',
            min_date_allowed=engagements['Date'].min(),
            max_date_allowed=engagements['Date'].max(),
            initial_visible_month=engagements['Date'].max(),
            start_date=engagements['Date'].min(),
            end_date=engagements['Date'].max(),
            className='date-picker',
            ),
        dbc.Row(
            [
            dcc.Markdown('**Overview** ', className='heading1'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/impression.png', className='icon')
                ),
                html.Div(id='impressions', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/engagements-icon.png', className='icon')
                ),
                html.Div(id='engagements', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/engagement.png', className='icon')
                ),
                html.Div(id='engagement_rate', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/like.png', className='icon')
                ),
                html.Div(id='emv_value', className='overview-container')
            ], 
            className='overview-wrapper'),
            html.Div([
                html.Div(
                    html.Img(src='/assets/img/followers.png', className='icon')
                ),
                html.Div(id='followers',children=['Followers', html.Br(), html.B("{:,.0f}".format(total_followers), className='overview-count')], className='overview-container')
            ], 
            className='overview-wrapper'),

        ]),

        dbc.Row([
            dcc.Markdown('**Follower Growth Trend** ', className='heading1'),
            html.P('Follower Growth shows the increase in the number of your followers. ', className='heading2'),

            dcc.Graph(
                figure=px.line(monthly_followers, x='YearMonth', y='New followers', color_discrete_sequence=['#b51a00'],  markers=True),
                className='followers-graph',
            ),

            dbc.Row([
                dcc.Dropdown(
                    id='audience-dropdown',
                    options=[
                        {'label': 'Job titles', 'value': 'Job titles'},
                        {'label': 'Locations', 'value': 'Locations'},
                        {'label': 'Industries', 'value': 'Industries'},
                        {'label': 'Seniority', 'value': 'Seniority'},
                        {'label': 'Company size', 'value': 'Company size'},
                        {'label': 'Companies', 'value': 'Companies'},
                    ],
                    value='Job titles',
                    className ='audience-dropdown'), 

                dcc.Graph(
                    id='audience-graph',
                    figure = px.bar(demographics[demographics['Top Demographics'] == 'Industries'], x='Percentage', y='Value', title=f'Top Industries by Percentage', labels={'Percentage': 'Percentage (%)', 'Value': 'Top Values'}).update_layout(plot_bgcolor='white'),
                    className='audience-graph')
                ], 
                className='audience-wrapper'
                ),
        ]),

        dbc.Row([
            dcc.Markdown('**Reach/Reactions Trend** ', className='heading1'),
            html.P('Follower Growth shows the increase in the number of your followers. ', className='heading2'),
            dcc.Dropdown(
                id='variable-dropdown',
                options=[
                    {'label': 'Engagements', 'value': 'Engagements'},
                    {'label': 'Impressions', 'value': 'Impressions'},
                    {'label': 'Engagement Rate', 'value': 'EngagementRate'},
                ],
                value='Engagements',
                className='variable-dropdown',),

            dbc.Row([    
                dcc.Graph(
                    id='engagements-graph',
                    figure=px.line(engagements, x='Date', y='Impressions', color_discrete_sequence=['#b51a00']),
                    className='engagements-graph'
                    ),
                
                dbc.Row([
                    dcc.Graph(
                        id='optimal-graph',
                        figure=px.histogram(engagements_by_day, x='DayOfWeek', y='Engagements'),
                        className = 'optimal-graph',
                        ),

                    html.Div([ 
                        html.B('What is the best day of week to post? \n', style={'margin-top': '0px', 'margin-right': '20px', 'Margin-left':'100px','text-align': 'center', 'color': 'gray', 'font-family': 'Helvetica Neue'}),
                        html.B(id ='best-day', children='Reach or Engagement', style={'margin-top': '20px', 'margin-right': '10px', 'font-size':'20px','Margin-left':'10px','align-self': 'center', 'color': 'black', 'font-family': 'Helvetica Neue'}),
                        html.P(id ='optimal-days',children=["Optimal day:", html.B('optimal_day', style={'font-size':'1000px'})], style={'margin-top': '10px', 'margin-right': '20px', 'Margin-left':'10px', 'color': 'Black', 'font-family': 'Arial, sans-serif'})],
                        style={'width':'400px','height':'200px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'10px'})
                ], style = {'width':'500px',})
            ]),  
        ]),
        dcc.Dropdown(
            id='posts-dropdown',
            options=[
                {'label': 'Top engaging posts', 'value': 'Top engaging posts'},
                {'label': 'Top posts by reach', 'value': 'Top posts by reach'},
            ],
            value='Top engaging posts',  
            style={'margin-top': '20px', 'width': '400px', 'margin-left': '40px', 'margin-right': '100px'}),

        dbc.Table(
            id = 'top-posts-preview',
            children=[
                html.Tr([
                    html.Th("Published on"),
                    html.Th("Posts"),
                    html.Th(""),
                    html.Th("Impressions"),
                    html.Th("Reactions"),
                    html.Th("Post Link"),
                    html.Th("Topics"),
                ], style={'background-color': 'lightgray'}),  # Header row styling
            ] +
            # Rows
            [html.Tr([
                html.Td(row['Post publish date'], style={'width':'200px', 'margin-left': '10px'}),
                html.Td(html.Img(src=row['Thumbnail'], height=100) if row['Thumbnail'] else html.P('Thumbnail not found.')),
                html.Td(row['Description'],style={'width':'800px', 'margin-left': '10px'}),
                html.Td(row['Impressions'], style={'width':'200px', 'margin-left': '10px'}),
                html.Td(row['Engagements'],style={'width':'200px',}),
                html.Td(html.A("View on LI", href=row['Post URL']), style={'width':'200px',}),
                html.Td(row['Topics'],style={'width':'300px',}),
            ], style={'border-bottom': '1px solid gray', 'padding': '10px','width':'1700px'}) for _, row in top_posts_preview_with_topics.head().iterrows()],
            bordered=True,
            responsive=True,
            striped=True,
            hover=True,
            style={'margin': '20px', 'width': '100%'}  # Overall table styling
        ), 
            
    ],
    className='app-container'
)

@app.callback(
    Output('engagements', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_engagements(start_date, end_date):
    return callbacks.update_engagements(start_date, end_date)

@app.callback(
    Output('impressions', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)    
def update_impressions(start_date, end_date):
    return callbacks.update_impressions(start_date, end_date)

@app.callback(
    Output('engagement_rate', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
) 
def update_engagements_rate(start_date, end_date):
    return callbacks.update_engagements_rate(start_date, end_date)

@app.callback(
    Output('emv_value', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)   
def update_emv(start_date, end_date):   
    return callbacks.update_emv(start_date, end_date)

@app.callback(
    Output('audience-graph', 'figure'),
    [Input('audience-dropdown', 'value')]
)
def update_audience_graph(selected_variable):
    return callbacks.update_audience_graph(selected_variable)

@app.callback(
    Output('engagements-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_engagements_graph(start_date, end_date, selected_variable):
    return callbacks.update_engagements_graph(start_date, end_date, selected_variable)

@app.callback(
    Output('optimal-graph', 'figure'),
    Output('best-day', 'children'),
    Output('optimal-days', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_optimal_graph(start_date, end_date, selected_variable):
    return callbacks.update_optimal_graph(start_date, end_date, selected_variable)


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
